# Explorando o uso API

### Recuperar estações automáticas (T)

Recuperar todas estações de acordo com o tipo passado como parâmetro:<br>
**URL BASE**:  https://apitempo.inmet.gov.br/estacoes/{type_station}<br>
**type_station (automatica)**: T <br>
**type_station (manual)**: M

In [2]:
import requests

# Obter os dados de uma url
def get_data(url):
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        
    return data


#  retorna dados de um tipo de estacão 
def estation_type(type_estation):
    
    url = f'https://apitempo.inmet.gov.br/estacoes/{type_estation}'
    
    data = get_data(url)
    
    return data


# concatenar os dados das estações Automaticas e Manauais
def get_all_stations():
    
    estacoes_automaticas = estation_type('T')
    estacoes_manuais = estation_type('M')

    todas_estacoes = []
    todas_estacoes.extend(estacoes_automaticas)
    todas_estacoes.extend(estacoes_manuais)
    
    return todas_estacoes

### Funções responsáveis por salvar dados em arquivos JSON e CSV

In [3]:
import json

def export_json(data, file_name):
    json_str = json.dumps(data, default=str)

    with open(f'../data/{file_name}.json', 'w') as file:
        file.write(json_str)
        

In [4]:

import csv

def export_csv(data, file_name):
    colunas = data[0].keys()

    with open(f'../data/{file_name}.csv', 'w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=colunas)      
        writer.writeheader()
        
        for linha in data:
            writer.writerow(linha)

### Salvar dados da API

In [5]:
# Obter os dados da API

todas_estacoes = get_all_stations()

### Exportar para JSON

In [6]:
export_json(todas_estacoes, 'data_raw_all_stations')

### Gerar dataframe

In [7]:
import pandas as pd

df = pd.read_json('../data/data_raw_all_stations.json')

# Transformação de dados:
#### 1 - Exluir colunas ['SG_ENTIDADE']<br>

In [8]:
df.drop('SG_ENTIDADE', axis=1, inplace=True)

#### 2 - Tratar valores Nan nas colunas ['DT_FIM_OPERACAO']

In [9]:
#1 - Como todos os valores são Nan
df.isna().sum()

#2 - Vou atualizar todos os campos para zero(0) e mudar o Dtype para int64
df['DT_FIM_OPERACAO'] = df['DT_FIM_OPERACAO'].fillna(0).astype('int64')

#### 3 - Preencher os valores Nan da coluna ['FL_CAPITAL'] com o valor 'N'


In [10]:
#1 - levando em consideração que a maioria estações não são capitais, vou atualizar os valores Nans com o valor 'N'

df['FL_CAPITAL'].value_counts()

#N    585
#S     18
#Nan  111
#Name: FL_CAPITAL, dtype: int64


df['FL_CAPITAL'] = df['FL_CAPITAL'].fillna('N')

 #### 4 - Formatar a coluna ['DT_INICIO_OPERACAO'] para exibir apenas a data no formato YYYY-MM-DD

In [11]:
# Come é:
# format: 2008-07-20T21:00:00.000-03:00

# como será:
# format: 2008-07-20

# aplicar split() na string.
df['DT_INICIO_OPERACAO'] = df['DT_INICIO_OPERACAO'].str.split('T').str[0]


## Salvar os dados:
#### 1 - formato JSON.<br>

In [14]:
# O parâmetro orient='records' indica que a lista de dicionários será preservada
# O parâmetro lines=True cria um arquivo JSON de linhas, onde cada linha é um dicionário

df.to_json('../data/data_processed_all_stations.json',orient='records')
#df.to_json('../data/data_processed_all_stations.json',orient='records', lines=True)

### Descrição das colunas

**CD_OSCAR** : 'Código internacionalmente reconhecido para a estação meteorológica'  
**DC_NOME** : 'Nome da estação meteorológica'  
**FL_CAPITAL** : 'Indicação se a estação é a capital do estado ("S" para sim, "N" para não)'  
**DT_FIM_OPERACAO** : 'Data de encerramento das operações da estação (pode ser nulo se ainda estiver em operação)'  
**CD_SITUACAO** : 'Situação atual da estação meteorológica (por exemplo, "Pane")'  
**TP_ESTACAO** : 'Tipo da estação meteorológica (por exemplo, "Automática")'  
**VL_LATITUDE** : 'Valor da latitude geográfica da estação'  
**CD_WSI** : 'Código único atribuído pela WSI (Weather Services International)'  
**CD_DISTRITO** : 'Código do distrito ao qual a estação pertence'  
**VL_ALTITUDE** : 'Altitude da estação meteorológica'  
**SG_ESTADO** : 'Sigla do estado onde a estação está localizada'  
**SG_ENTIDADE** : 'Sigla da entidade responsável pela estação meteorológica (por exemplo, "INMET")'  
**CD_ESTACAO** : 'Código único atribuído à estação'  
**VL_LONGITUDE** : 'Valor da longitude geográfica da estação'  
**DT_INICIO_OPERACAO** : 'Data de início das operações da estação meteorológica'  
